This notebook is part of the [Machine Learning class](https://github.com/erachelson/MLclass) by [Emmanuel Rachelson](https://personnel.isae-supaero.fr/emmanuel-rachelson?lang=en) and was written by Erwan Lecarpentier and Jonathan Sprauel.

License: CC-BY-SA-NC.

<div style="font-size:22pt; line-height:25pt; font-weight:bold; text-align:center;">XGBoost<br>Introduction to XGBoost</div>



[XGBoost](https://xgboost.readthedocs.io/en/latest/) is a library. It implements machine learning algorithms (Figure 1) that are all working with the [gradient boosting](https://en.wikipedia.org/wiki/Gradient_boosting) framework. It can be used for regression and classification. It produces efficient models to deal with standard tabular data as opposed to more fancy data structures like images, sounds, videos etc.

<img id="fig1" src="img/machine_learning.png">
<center>Figure 1: a machine learning algorithm</center>


This Practice Course is composed of 3 parts - each part is meant to be done in about 1 hour :
* In the **first notebook**, you will learn the **basic of XGBoost**, how to apply it on a dataset and tune it to obtain the best performances.
* In the **second notebook**, we will focus on **ensemble methods** and explain what makes XGBoost different from other models.
* Finally in the **last notebook** you will see how the choice of a method (such as XGBoost) is a key element of a tradeoff between **Bias and Variance**. 


# <a id="sec1"></a> What is XGBoost?

XGBoost (eXtreme Gradient Boosting) is a library for Gradient Tree Boosting in C++, Java, Python, R and Julia, that was initially developped by Tianqi Chen.
XGBoost has recently been dominating applied machine learning and Kaggle competitions for structured or tabular data. For reference (and inspiration), you can have a look at this curated list of first, second and third place competition [winners that used XGBoost](https://github.com/dmlc/xgboost/tree/master/demo#machine-learning-challenge-winning-solutions).

> When in doubt, use xgboost.
> [Avito Winner’s Interview](http://blog.kaggle.com/2015/08/26/avito-winners-interview-1st-place-owen-zhang/)


XGBoost has been designed for speed and performance : it is usually faster than sklearn's GradientBoost, even though they are fundamentally the same as they are both gradient boosting implementations.

## Installing XGBoost

With Anaconda use :
```conda install -c anaconda py-xgboost```

With pip use :
```pip install xgboost```


!pip install xgboost -U

# Simple XGBoost usage

Before we begin, be sure that you have the following dependencies and run the import:

In [ ]:
# !pip install seaborn numpy pandas sklearn matplotlib

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib import style
import seaborn as sns
sns.set_style('whitegrid')

from sklearn import datasets
from sklearn.model_selection import train_test_split, cross_val_predict
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import SGDClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.decomposition import PCA
from sklearn.preprocessing import LabelEncoder, MinMaxScaler
from sklearn.metrics import confusion_matrix, precision_score, recall_score, f1_score
from sklearn.model_selection import cross_val_score
import time
import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)
%matplotlib inline


from xgboost import XGBClassifier
import xgboost as xgb

XGboost natively uses a specific format, that is optimized in term of memory and computation speed : DMatrix.
When using numpy, you have to convert it explicitely.

In [ ]:
iris = datasets.load_iris()
X_train, X_test, y_train, y_test = train_test_split(iris.data, iris.target, test_size=0.2, random_state=42)

dtrain = xgb.DMatrix(X_train, label=y_train)
dtest = xgb.DMatrix(X_test, label=y_test)


The major parameters are as following - we will focus on the parameters later on.

In [ ]:
param = {
    'max_depth': 3,  # the maximum depth of each tree
    'eta': 0.3,  # Learning rate, the training step for each iteration
    'silent': 1,  # logging mode - quieter and faster
    'objective': 'multi:softprob',  # error evaluation for multiclass training
    'num_class': 3}  # the number of classes that exist in this dataset
num_round = 20  # the number of training iterations

Finally, we run the training and prediction with an api that is similar to sklearn - though we have to the true prediction : for each line, we must select the class where the probability is the highest.

In [ ]:
%%timeit

bst = xgb.train(param, dtrain, num_round)
preds = bst.predict(dtest)
predictions = np.asarray([np.argmax(line) for line in preds])
#print (precision_score(y_test, predictions, average='macro')) # 1.0

With a bit less performance, you can also directly use the pandas-compatible functions that mirror exactly the sklearn api.

In [ ]:
%%timeit

mod = XGBClassifier(**param)
mod.fit(X_train, y_train)
predictions = mod.predict(X_test)
#print (precision_score(y_test, predictions, average='macro'))

Finally, you can save the model either with pickle or with the dedicated function :

In [ ]:
bst = xgb.train(param, dtrain, num_round)
preds = bst.predict(dtest)

bst.save_model('xgb.model')

bst2 = xgb.Booster(model_file='xgb.model')

preds2 = bst2.predict(dtest)
# assert they are the same
assert np.sum(np.abs(preds2 - preds)) == 0

# A complete exemple : Classification of stars, Galaxies, Quasars

For this first application of XGBoost, we will try to classify observations of space to be either stars, galaxies or quasars.
We are using data from the [Sloan Digital Sky Survey](http://www.sdss.org/)

### About the SDSS
The Sloan Digital Sky Survey is a project which offers public data of space observations. Observations have been made since 1998 and have been made accessible to everyone who is interested.

For this purpose a special 2.5 m diameter telescope was built at the Apache Point Observatory in New Mexico, USA. The telescope uses a camera of 30 CCD-Chips with 2048x2048 image points each. The chips are ordered in 5 rows with 6 chips in each row. Each row observes the space through different optical filters (u, g, r, i, z) at wavelengths of approximately 354, 476, 628, 769, 925 nm.

The telescope covers around one quarter of the earth's sky - therefore focuses on the northern part of the sky.

In [ ]:
sdss_df = pd.read_csv('Skyserver_SQL2_27_2018 6_51_39 PM.csv', skiprows=1)
sdss_df.head()

In [ ]:
sdss_df.describe()

In the previous cells, we only checked a few classical elements when facing a dataset : 
* There is no missing data, that we should complete
* Most features remain within reasonable values, for each columns

The goal is to classify each data into either the Galaxy, Star or QSO class.

In [ ]:
sdss_df['class'].value_counts()

## Data Analysis

Before applying any classification algorithm, let's look a bit more and transform the data : first we remove the column that obviously won't help classify into the correct class, such as the objects id and parameters of the camera at the moment of observation.

In [ ]:
sdss_df.drop(['objid', 'run', 'rerun', 'camcol', 'field', 'specobjid'], axis=1, inplace=True)
sdss_df.head(1)

Next we look at a few interesting features (univariate analysis) : by plotting the distribution of each class along this feature, we can estimate if this feature can help in classifying the data.

For instance, we can see that redshift seems to have good correlation, while ascension and declination does not differ significantly between the 3 classes.

In [ ]:
fig, axes = plt.subplots(nrows=1, ncols=3,figsize=(16, 4))
ax = sns.distplot(sdss_df[sdss_df['class']=='STAR'].redshift, bins = 30, ax = axes[0], kde = False)
ax.set_title('Star')
ax = sns.distplot(sdss_df[sdss_df['class']=='GALAXY'].redshift, bins = 30, ax = axes[1], kde = False)
ax.set_title('Galaxy')
ax = sns.distplot(sdss_df[sdss_df['class']=='QSO'].redshift, bins = 30, ax = axes[2], kde = False)
ax = ax.set_title('QSO')

In [ ]:
sns.lmplot(x='ra', y='dec', data=sdss_df, hue='class', fit_reg=False, palette='coolwarm', height=6, aspect=2)
plt.title('Equatorial coordinates')

Finally, we transform a few features : we transform the different bands through a PCA, we encode the classes and scale the extreme values.

In [ ]:
sdss_df_fe = sdss_df

# encode class labels to integers
le = LabelEncoder()
y_encoded = le.fit_transform(sdss_df_fe['class'])
sdss_df_fe['class'] = y_encoded

# Principal Component Analysis
pca = PCA(n_components=3)
ugriz = pca.fit_transform(sdss_df_fe[['u', 'g', 'r', 'i', 'z']])

# update dataframe 
sdss_df_fe = pd.concat((sdss_df_fe, pd.DataFrame(ugriz)), axis=1)
sdss_df_fe.rename({0: 'PCA_1', 1: 'PCA_2', 2: 'PCA_3'}, axis=1, inplace = True)
sdss_df_fe.drop(['u', 'g', 'r', 'i', 'z'], axis=1, inplace=True)
sdss_df_fe.head()

In [ ]:
scaler = MinMaxScaler()
sdss = scaler.fit_transform(sdss_df_fe.drop('class', axis=1))

## Classification

Using XGboost is similar in many ways with the sklearn api : we define a model and call the *fit* function on the training data; the *predict* function makes the prediction.

<div class="alert alert-success">
<b>Exercice 1 :</b> Complete the following functions.
</div>

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(..., test_size=0.33)

In [ ]:
%%time
xgbC = XGBClassifier(n_estimators=100)
xgbC.fit(...)
preds = xgbC.predict(...)
acc_xgb = (preds == y_test).sum().astype(float) / len(preds)*100
print("XGBoost's prediction accuracy is: %3.2f" % (acc_xgb))


In [ ]:
%%time
rfc = RandomForestClassifier(n_estimators=100)
...
acc_rfc = (preds == y_test).sum().astype(float) / len(preds)*100
print("Scikit-Learn's Random Forest Classifier's prediction accuracy is: %3.2f" % (acc_rfc))

In [ ]:
%%time
svc = SVC()
...
acc_svc = (preds == y_test).sum().astype(float) / len(preds)*100
print("Scikit-Learn's Support Vector Machine Classifier's prediction accuracy is: %3.2f" % (acc_svc))

We can focus a bit more on the performance comparison between XGBoost and Random Forest, in term of optimality :

In [ ]:

rfc_cv = RandomForestClassifier(n_estimators=100)
scores = cross_val_score(rfc_cv, X_train, y_train, cv=10, scoring = "accuracy")
print("Scores:", scores)
print("Mean:", scores.mean())
print("Standard Deviation:", scores.std())

In [ ]:
xgb_cv = XGBClassifier(n_estimators=100)
scores = cross_val_score(xgb_cv, X_train, y_train, cv=10, scoring = "accuracy")
print("Scores:", scores)
print("Mean:", scores.mean())
print("Standard Deviation:", scores.std())

Finally, XGBoost also allows to obtain the feature importance list :

In [ ]:
importances = pd.DataFrame({
    'Feature': sdss_df_fe.drop('class', axis=1).columns,
    'Importance': xgbC.feature_importances_
})
importances = importances.sort_values(by='Importance', ascending=False)
importances = importances.set_index('Feature')
importances

# A focus on the parameters

XGBoost has <[many parameters](https://xgboost.readthedocs.io/en/latest/parameter.html); we will explain them a bit by following this guideline, that woks well on most problems :

* Choose a relatively high learning rate (**learning_rate**)
* Determine the optimum number of trees for this learning rate. (**n_estimators**)
* Tune tree-specific parameters ( **max_depth, min_child_weight, gamma, subsample, colsample_bytree** for instance) for the decided learning rate and number of trees.
* Tune regularization parameters (**lambda, alpha**)

For most of the functions, instead of relying on sklearn cross_valisation function we will use XGBoost function called "cv" which performs cross-validation at each boosting iteration and returns the optimum number of trees required.

<div class="alert alert-success">
<b>Exercice 2 :</b> Complete the following functions.
</div>





In [ ]:
num_round = 100
param = {
    "learning_rate" :0.3,
    "n_estimators":1000,
    'silent': 1,
    'objective': 'multi:softprob',
    'num_class': 3
}

...

cvresult = xgb.cv(param, dtrain, nfold=10, num_boost_round=param['n_estimators'],  early_stopping_rounds=50)

bst = xgb.train(param, dtrain, num_round)
preds = bst.predict(dtest)
predictions = np.asarray([np.argmax(line) for line in preds])
print (precision_score(y_test, predictions, average='macro'))

In [ ]:
print(cvresult.shape)


<div class="alert alert-success">
<b>Exercice 3 :</b> Complete the following code to tune the tree parameters : max_depth and min_child_weight
</div>

In [ ]:
from sklearn.model_selection import GridSearchCV

param = {
    "learning_rate" :0.3,
    "n_estimators":...,
    "max_depth":7,
    "min_child_weight":2,
    'objective': 'multi:softprob',
    'num_class': 3
}

param_test1 = {
 'max_depth':...,
 'min_child_weight':...
}
gsearch1 = GridSearchCV(estimator = XGBClassifier(param), param_grid = param_test1, cv=10)

gsearch1.fit(X_train, y_train)
gsearch1.cv_results_, gsearch1.best_params_, gsearch1.best_score_

The process can be repeated iteratively with the remaining parameters. A good heuristic is to follow this order next :
Gamma, then (subsample and colsample_bytree) together, then reg_lambda and reg_alpha.


<div class="alert alert-success">
<b>Exercice 3 :</b> Complete the tuning and try to obtain the best performance of the model.
</div>

In [ ]:

# replace parameters for final evaluation
param = {
    "learning_rate" :0.1,
    "n_estimators":76,
    "max_depth":6,
    "min_child_weight":1,
    "gamma":0,
    "subsample":0.8,
    "colsample_bytree":0.8,
    "nthread":4,
    "scale_pos_weight":1,
    'silent': 1,
    'objective': 'multi:softprob',
    'num_class': 3
}


bst = xgb.train(param, dtrain, num_round)
preds = bst.predict(dtest)
predictions = np.asarray([np.argmax(line) for line in preds])
print (precision_score(y_test, predictions, average='macro'))


# Conclusion of the notebook

In the notebook, we have seen the basic functions to use XGBoost. 
The next two notebooks will be more open-ended : the first one will be focused on the ensemble methods, while the second one will be focused on the tuning of the parameters, with regards to the tradeoff between biais and variance.

Sources : 
* https://www.kaggle.com/lucidlenn/data-analysis-and-classification-using-xgboost
* https://github.com/dmlc/xgboost/tree/master/demo